Importing the libraries

In [148]:
import torch
import torch.nn as nn
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch import optim
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

Importing the dataset

In [149]:
train_dataset=(r'/Users/namanparuthi/Desktop/Rock Paper Scissors SXSW-3/train',
               r'/Users/namanparuthi/Desktop/Rock Paper Scissors SXSW-3/train/_annotations.csv',
                             
                             
                             )

test_dataset=(r'/Users/namanparuthi/Desktop/Rock Paper Scissors SXSW-3/test',
              r'/Users/namanparuthi/Desktop/Rock Paper Scissors SXSW-3/test/_annotations.csv',
                             
                         
                             )


In [150]:
train_loader=DataLoader(train_dataset,batch_size=10,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=10,shuffle=True)

Defining the object detecton model(R-CNN)

In [151]:
def create_object_detection_model(num_classes):
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

Defining the classification CNN(custom cnn)

In [152]:
class ClassificationCNN(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        


Combining the detection and classification models

In [163]:
class CombinedModel(nn.Module):
    def __init__(self, num_classes_detection, num_classes_classification):
        super(CombinedModel, self).__init__()
        self.object_detection_model = create_object_detection_model(num_classes_detection)
        self.classification_model = CombinedModel(num_classes_classification)

    def forward(self, images):
        # Object Detection
        detection_output = self.object_detection_model(images)
        boxes = detection_output[0]['boxes']
        # Crop and classify detected objects
        object_images = [F.crop(images[i], int(box[1]), int(box[0]), int(box[3]) - int(box[1]), int(box[2]) - int(box[0])) for i, box in enumerate(boxes)]
        object_images = torch.stack(object_images)
        classification_output = self.classification_model(object_images)
        return detection_output, classification_output


Training the model

In [154]:
import datetime

def training_loop(n_epochs,optimizer,model,loss_fn,train_loader):
    for epoch in range(1,n_epochs+1):
        loss_train=0.0
        loss_cmu=0
        for image,label in train_loader:
            outputs=model(image)
            loss=loss_fn(outputs,label)
            loss_train=0
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
            print(loss_train)
            loss_cmu+=loss_train

        if epoch==1 or epoch%10==0: 
            print(f'{datetime.datetime.now()} Epoch {epoch}, train loss={loss_cmu}')
            

In [155]:
model=CombinedModel(3,3)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0.4)
loss_fn = nn.CrossEntropyLoss()



In [156]:
model=CombinedModel(3,3)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
loss_fn = nn.CrossEntropyLoss()



In [157]:
def training_loop2(n_epochs,optimizer,model,loss_fn,train_loader):
    for epoch in range(n_epochs):
    # Train
        for batch_idx, data, target in enumerate(train_loader):
            optimizer.zero_grad()
            data=data
            target=target
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
        
        # Print progress
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [158]:
model=CombinedModel(3,3)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
loss_fn = nn.CrossEntropyLoss()

